# Model #2 for image prediction

In [1]:
import numpy as np
import pandas as pd
# from skimage import imread
from matplotlib.image import imread
import matplotlib.pyplot as plt
import os
import cv2
import random
import pickle
from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models,callbacks
import sys
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
# sys.path.append(os.path.abspath(r'C:\Users\raulg\Documents\THEBRIDGE_DS\0.-Repo_Git\ml_alzheimer_class\src'))
sys.path.append(os.path.relpath('../src'))
import utils as ut
import math


## Data loading and mix

The baseline model had some funny results - the validation results were quite bad (0.37 accuracy), but the test results reached a 0.87 in accuracy. I'll join the arrays and make a random split

In [2]:
x1_train,x1_test,y1_train,y1_test = ut.img_images_load()
print(x1_train.shape)
print(x1_test.shape)
print(y1_train.shape)
print(y1_test.shape)

(33984, 32, 32, 1)
(6400, 32, 32, 1)
(33984,)
(6400,)


In [12]:
6400/40384

0.15847860538827258

In [14]:
x1 = np.concatenate((x1_train,x1_test))
y1 = np.concatenate((y1_train,y1_test))
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1,test_size=.15,stratify=y1,random_state=42)
print('x1_train shape', x1_train.shape)
print('x1_test shape', x1_test.shape)
print('y1_train shape', y1_train.shape)
print('y1_test shape', y1_test.shape)
unique_train, counts_train = np.unique(y1_train, return_counts=True)
unique_test, counts_test = np.unique(y1_test, return_counts=True)
print('y1_train distribution: \n',np.asarray((unique_train, counts_train)).T)
print('y1_test distribution: \n',np.asarray((unique_test, counts_test)).T)


x1_train shape (34326, 32, 32, 1)
x1_test shape (6058, 32, 32, 1)
y1_train shape (34326,)
y1_test shape (6058,)
y1_train distribution: 
 [[    0 10880]
 [    1  9520]
 [    2  8377]
 [    3  5549]]
y1_test distribution: 
 [[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]


In [4]:
### Test whether stratify is ok
unique_train, counts_train = np.unique(y1_train, return_counts=True)
unique_test, counts_test = np.unique(y1_test, return_counts=True)
print(np.asarray((unique_train, counts_train)).T)
print(np.asarray((unique_test, counts_test)).T)

[[    0 10880]
 [    1  9520]
 [    2  8377]
 [    3  5549]]
[[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]


Copy of the baseline model. I'll check the results now that the sets have been randomly mixed

In [23]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

# Capa densa con Dropout
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# label_encoder = LabelEncoder()
# y1_train_encoded = label_encoder.fit_transform(y1_train)
# y2_test_encoded = label_encoder.transform(y1_test)
model.fit(x1_train,y1_train,epochs=70,batch_size = 128,validation_split = 0.2, callbacks=callbacks.EarlyStopping(patience=10)
          )

c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.3406 - loss: 1.3291 - val_accuracy: 0.5763 - val_loss: 1.0638
Epoch 2/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4370 - loss: 1.1318 - val_accuracy: 0.6129 - val_loss: 0.9307
Epoch 3/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4823 - loss: 1.0356 - val_accuracy: 0.6228 - val_loss: 0.8411
Epoch 4/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4998 - loss: 0.9816 - val_accuracy: 0.6579 - val_loss: 0.7825
Epoch 5/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5120 - loss: 0.9423 - val_accuracy: 0.6668 - val_loss: 0.7651
Epoch 6/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5231 - loss: 0.9280 - val_accuracy: 0.6899 - val_loss: 0.7234
Epoch 7/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5309 - loss: 0.8997 - val_accuracy: 0.7017 - val_loss: 0.6878
Epoch 8/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.5644 - loss: 0.8863 - val_accu

In [29]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 30, 30, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │        73,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,126 (1.06 MB)

 Trainable params: 92,708 (362.14 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 185,418 (724.29 KB)

In [ ]:
model.evaluate(x1_test, y1_test)

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8146 - loss: 0.5198


[0.5098863840103149, 0.8189171552658081]

In [27]:
confusion_matrix(y1_test, model.predict(x1_test).argmax(axis=1),normalize='pred') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[0.81402142, 0.130255  , 0.03421562, 0.0059761 ],
       [0.17234664, 0.73742247, 0.15816656, 0.01095618],
       [0.01363194, 0.130255  , 0.80503551, 0.01494024],
       [0.        , 0.00206754, 0.00258231, 0.96812749]])

The new model performs generally poorer than the baseline, but at least now we're sure that it is more robust as we know now that both train and test include colored images.

In [68]:
model.save("../models/image/baseline_model_a.keras")


## 2nd model - with rotation, zoom in/out of images

Three changes are considered from the baseline model:
- The accuracies of the train, validation and test (0.67,0.81,0.81) don't seem to signal overfitting. I'll make no use of the Dropout layer
- Accuracy musn't be our main metric: we prefer false positives, and rather be extremely sure that the model doesn't miss any alzheimer case.
- Lastly, we'll make use of an image data generator, so the model has more differences to check.

In [8]:
### Datagen with validation split is not supported, so the split must be done prior to the model:
x2_train = x1_train
x2_test = x1_test
y2_train = y1_train
y2_test = y1_test
x2_train,x2_val,y2_train,y2_val = train_test_split(x2_train,y2_train,test_size=.2,stratify=y2_train,random_state=42)
print(x2_train.shape)
print(x2_test.shape)
print(y2_train.shape)
print(y2_test.shape)
unique_train, counts_train = np.unique(y2_train, return_counts=True)
unique_test, counts_test = np.unique(y2_test, return_counts=True)
unique_val, counts_val = np.unique(y2_val, return_counts=True)
print(np.asarray((unique_train, counts_train)).T)
print(np.asarray((unique_test, counts_test)).T)
print(np.asarray((unique_val, counts_val)).T)

(27460, 32, 32, 1)
(6058, 32, 32, 1)
(27460,)
(6058,)
[[   0 8704]
 [   1 7616]
 [   2 6701]
 [   3 4439]]
[[   0 1920]
 [   1 1680]
 [   2 1479]
 [   3  979]]
[[   0 2176]
 [   1 1904]
 [   2 1676]
 [   3 1110]]


In [65]:
model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Flatten())

# Capa densa con Dropout
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model2.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# label_encoder = LabelEncoder()
# y1_train_encoded = label_encoder.fit_transform(y1_train)
# y2_test_encoded = label_encoder.transform(y1_test)
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotación aleatoria entre -20 y 20 grados
    width_shift_range=0.2,  # Desplazamiento horizontal aleatorio
    height_shift_range=0.2,  # Desplazamiento vertical aleatorio
    shear_range=0.2,  # Cortes aleatorios
    zoom_range=0.2,  # Zoom aleatorio
    horizontal_flip=True,  # Volteo horizontal
    fill_mode='nearest'  # Rellenar píxeles vacíos con el valor más cercano
)
model2.fit(datagen.flow(x2_train, y2_train, batch_size=128),epochs=70,validation_data=(x2_val, y2_val), callbacks=callbacks.EarlyStopping(patience=10)
          )

Epoch 1/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.3269 - loss: 1.3395 - val_accuracy: 0.3930 - val_loss: 1.2749
Epoch 2/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4027 - loss: 1.2458 - val_accuracy: 0.3456 - val_loss: 1.3052
Epoch 3/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4153 - loss: 1.2297 - val_accuracy: 0.3031 - val_loss: 1.3712
Epoch 4/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4150 - loss: 1.2195 - val_accuracy: 0.4059 - val_loss: 1.2288
Epoch 5/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.4306 - loss: 1.1849 - val_accuracy: 0.4800 - val_loss: 1.1496
Epoch 6/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - accuracy: 0.4407 - loss: 1.1663 - val_accuracy: 0.5124 - val_loss: 1.0678
Epoch 7/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4571 - loss: 1.1338 - val_accuracy: 0.5216 - val_loss: 1.0288
Epoch 8/70
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 50ms/step - accuracy: 0.4622 - loss: 1.1189 - 

In [16]:
model2.evaluate(x2_test, y2_test)

NameError: name 'model2' is not defined

In [67]:
confusion_matrix(y2_test, model2.predict(x2_test).argmax(axis=1),normalize='pred') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([[0.67056431, 0.28503788, 0.13718245, 0.00344828],
       [0.30045755, 0.46212121, 0.27713626, 0.00114943],
       [0.02897814, 0.24431818, 0.53718245, 0.00114943],
       [0.        , 0.00852273, 0.04849885, 0.99425287]])

In [70]:
model2.save("../models/image/model_2.keras")


The new model predicts in a worse manner, so ImageDataGenerator doesn't seem to be making a positive effect.

In [5]:
model1b = models.Sequential()
model1b.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)))
model1b.add(layers.MaxPooling2D((2, 2)))
model1b.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1b.add(layers.MaxPooling2D((2, 2)))
model1b.add(layers.Flatten())

# Capa densa con Dropout
model1b.add(layers.Dense(32, activation='relu'))
model1b.add(layers.Dropout(0.5))  # Dropout para reducir overfitting

model1b.add(layers.Dense(4, activation='softmax'))

# Compilar el modelo
model1b.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# label_encoder = LabelEncoder()
# y1_train_encoded = label_encoder.fit_transform(y1_train)
# y2_test_encoded = label_encoder.transform(y1_test)
model1b.fit(x2_train,y2_train,epochs=70,batch_size = 128,validation_data=(x2_val, y2_val), callbacks=callbacks.EarlyStopping(patience=10)
          )

Epoch 1/70


c:\Users\raulg\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


213/213 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.3511 - loss: 1.3023 - val_accuracy: 0.5856 - val_loss: 1.0276
Epoch 2/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.4749 - loss: 1.0654 - val_accuracy: 0.6335 - val_loss: 0.8748
Epoch 3/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.5108 - loss: 0.9836 - val_accuracy: 0.6649 - val_loss: 0.7888
Epoch 4/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5762 - loss: 0.9281 - val_accuracy: 0.6828 - val_loss: 0.7338
Epoch 5/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5812 - loss: 0.9036 - val_accuracy: 0.6953 - val_loss: 0.7026
Epoch 6/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.5985 - loss: 0.8660 - val_accuracy: 0.7183 - val_loss: 0.6705
Epoch 7/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.6100 - loss: 0.8101 - val_accuracy: 0.7103 - val_loss: 0.6472
Epoch 8/70
213/213 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.6195 - loss: 0.7923 - val_accuracy: 0.730

In [9]:
model1b.evaluate(x2_test, y2_test)

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8498 - loss: 0.4034


[0.3858424425125122, 0.8572136163711548]

In [17]:
recall_score(y2_test, model1b.predict(x2_test).argmax(axis=1),average='weighted')

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


np.float64(0.857213601848795)

In [16]:
confusion_matrix(y2_test, model1b.predict(x2_test).argmax(axis=1),normalize='pred') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


array([[0.76306342, 0.00361011, 0.00231616, 0.        ],
       [0.23454328, 0.99638989, 0.1447597 , 0.01409869],
       [0.0023933 , 0.        , 0.85292415, 0.        ],
       [0.        , 0.        , 0.        , 0.98590131]])

In [12]:
recall_score(y2_test, model1b.predict(x2_test).argmax(axis=1),average='weighted') #,normalize='pred'

190/190 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


np.float64(0.857213601848795)

In [20]:
model1b.save("../models/image/model_2.keras")


In [21]:
model1b = models.load_model("../models/image/model_2.keras")
